In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from google.colab import files  # Mengimpor modul `files` untuk mengelola file di Google Colab
import pandas as pd

In [2]:
# Upload dataset
uploaded = files.upload()
train_file = next(iter(uploaded.keys()))
train_data = pd.read_csv(train_file, header=None)

# Splitting features and target
X_train = train_data.iloc[:, :-1].values  # Features for training
y_train = train_data.iloc[:, -1].values  # Labels for training

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Split train-test
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

Saving optdigits.tra to optdigits.tra


In [3]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=12)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=12)

In [4]:
# Define RNN model to mimic HMM
class MarkovRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, pooling_type):
        super(MarkovRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        if self.pooling_type == 'max':
            out = torch.max(out, dim=1).values
        elif self.pooling_type == 'avg':
            out = torch.mean(out, dim=1)
        out = self.fc(out)
        return out

In [5]:
# Training function
def train_model(model, train_loader, optimizer, criterion, num_epochs, scheduler=None):
    model.train()
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch.unsqueeze(1))  # Add sequence dimension
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        if scheduler:
            scheduler.step()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

In [6]:
# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch.unsqueeze(1))  # Add sequence dimension
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [7]:
# Parameters
input_size = X_train.shape[1]
output_size = len(torch.unique(y_train_tensor))
hidden_sizes = [32, 64]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100, 250, 350]
optimizers = {'SGD': optim.SGD, 'RMSProp': optim.RMSprop, 'Adam': optim.Adam}

# Results dictionary
results = {
    'hidden_size': {},
    'pooling_type': {},
    'optimizer': {},
    'epochs': {}
}

In [8]:
# Experiment: Hidden Sizes
for hidden_size in hidden_sizes:
    print(f"\nExperimenting with Hidden Size: {hidden_size}")
    model = MarkovRNN(input_size, hidden_size, num_layers=1, output_size=output_size, pooling_type='avg')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=50, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['hidden_size'][hidden_size] = accuracy


Experimenting with Hidden Size: 32


Training Epochs:   2%|▏         | 1/50 [00:00<00:25,  1.92it/s]

Epoch 1/50, Loss: 0.6429


Training Epochs:   4%|▍         | 2/50 [00:00<00:20,  2.34it/s]

Epoch 2/50, Loss: 0.1251


...

Training Epochs:  98%|█████████▊| 49/50 [00:19<00:00,  2.56it/s]

Epoch 49/50, Loss: 0.0025


Training Epochs: 100%|██████████| 50/50 [00:19<00:00,  2.55it/s]

Epoch 50/50, Loss: 0.0025


Test Accuracy: 98.04%


In [9]:
# Experiment: Pooling Types
for pooling_type in pooling_types:
    print(f"\nExperimenting with Pooling Type: {pooling_type}")
    model = MarkovRNN(input_size, 32, num_layers=1, output_size=output_size, pooling_type=pooling_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=50, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['pooling_type'][pooling_type] = accuracy


Experimenting with Pooling Type: max


Training Epochs:   2%|▏         | 1/50 [00:00<00:19,  2.49it/s]

Epoch 1/50, Loss: 0.6704


Training Epochs:   4%|▍         | 2/50 [00:00<00:19,  2.48it/s]

Epoch 2/50, Loss: 0.1244


...

Training Epochs:  98%|█████████▊| 49/50 [00:19<00:00,  2.50it/s]

Epoch 49/50, Loss: 0.0063


Training Epochs: 100%|██████████| 50/50 [00:20<00:00,  2.49it/s]

Epoch 50/50, Loss: 0.0063


Test Accuracy: 96.73%


In [10]:
# Experiment: Optimizers
for optimizer_name, optimizer_class in optimizers.items():
    print(f"\nExperimenting with Optimizer: {optimizer_name}")
    model = MarkovRNN(input_size, 32, num_layers=1, output_size=output_size, pooling_type='avg')
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_class(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=50, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['optimizer'][optimizer_name] = accuracy


Experimenting with Optimizer: SGD


Training Epochs:   2%|▏         | 1/50 [00:00<00:17,  2.76it/s]

Epoch 1/50, Loss: 2.2182


Training Epochs:   4%|▍         | 2/50 [00:00<00:17,  2.73it/s]

Epoch 2/50, Loss: 1.9678


...

Training Epochs:  98%|█████████▊| 49/50 [00:20<00:00,  2.39it/s]

Epoch 49/50, Loss: 0.0055


Training Epochs: 100%|██████████| 50/50 [00:20<00:00,  2.42it/s]

Epoch 50/50, Loss: 0.0055


Test Accuracy: 97.39%


In [11]:
# Experiment: Epochs
for num_epochs in epochs_list:
    print(f"\nExperimenting with Epochs: {num_epochs}")
    model = MarkovRNN(input_size, 32, num_layers=1, output_size=output_size, pooling_type='avg')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=num_epochs, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['epochs'][num_epochs] = accuracy


Experimenting with Epochs: 5


Training Epochs:  20%|██        | 1/5 [00:00<00:01,  2.38it/s]

Epoch 1/5, Loss: 0.6502


Training Epochs:  40%|████      | 2/5 [00:00<00:01,  2.48it/s]

Epoch 2/5, Loss: 0.1244


...

Training Epochs: 100%|█████████▉| 349/350 [02:52<00:00,  1.98it/s]

Epoch 349/350, Loss: 0.0059


Training Epochs: 100%|██████████| 350/350 [02:52<00:00,  2.02it/s]

Epoch 350/350, Loss: 0.0058


Test Accuracy: 97.12%


In [12]:
# Display results
print("\nExperiment Results:")
for key, value in results.items():
    print(f"{key}: {value}")


Experiment Results:
hidden_size: {32: 0.9699346405228758, 64: 0.9803921568627451}
pooling_type: {'max': 0.9673202614379085, 'avg': 0.9673202614379085}
optimizer: {'SGD': 0.9006535947712418, 'RMSProp': 0.9699346405228758, 'Adam': 0.9738562091503268}
epochs: {5: 0.9725490196078431, 50: 0.9660130718954248, 100: 0.9686274509803922, 250: 0.9751633986928104, 350: 0.9712418300653595}
